**This code is an example of loading necessary information from the 2 VIIRS files as model input.**

Step 1: Import Python Libs

In [ ]:
from netCDF4 import Dataset
import os, glob
import numpy as np
import h5py

In [ ]:
from google.colab import drive
drive.mount('content/')

Drive already mounted at content/; to attempt to forcibly remount, call drive.mount("content/", force_remount=True).


Step 2: Mount Google Drive and define VIIRS files

In [ ]:
#here I just give the VIIRS location:
# v02_path = '/content/content/My Drive/Colab Notebooks/Chenxi_Shared_Research/Data/VIIRS/'
# v03_path = '/content/content/My Drive/Colab Notebooks//Chenxi_Shared_Research/Data/VIIRS/'
# v02_path = '/content/drive/MyDrive/Chenxi_Shared_Research/Data/VIIRS/'
# v03_path = '/content/drive/MyDrive/Chenxi_Shared_Research/Data/VIIRS/'
v02_path = '/content/content/My Drive/Colab Notebooks/chenxi/Data/VIIRS/'
v03_path = '/content/content/My Drive/Colab Notebooks/chenxi/Data/VIIRS/'
# data = np.load(v03_path + 'VNP03MOD.A2014187.0200.001.2017255041028.nc')

viirs_timeflag = '2014187.0200'
v02_file = glob.glob(v02_path+'VNP02*'+viirs_timeflag+'*.nc')[0]
v03_file = glob.glob(v03_path+'VNP03*'+viirs_timeflag+'*.nc')[0]

Step 3: Load VIIRS Geolocation file (VNP03MOD)

*You will see the dimensions of each dataset are 3232 x 3200, where 3232 is the along-track dimension, and 3200 is the cross-track dimension.*

In [ ]:
v03 = Dataset(v03_file,'r')
v03_lon = v03['/geolocation_data/longitude'][:]
v03_lat = v03['/geolocation_data/latitude'][:]
v03_sza = v03['/geolocation_data/solar_zenith'][:]
v03_vza = v03['/geolocation_data/sensor_zenith'][:]
v03_saa = v03['/geolocation_data/solar_azimuth'][:]
v03_vaa = v03['/geolocation_data/sensor_azimuth'][:]
v03.close()


print (v03_sza)
print (v03_sza.shape)

[[36.85     36.84     36.829998 ... 20.99     20.98     20.98    ]
 [36.86     36.85     36.829998 ... 21.       21.       21.      ]
 [36.87     36.86     36.84     ... 21.01     21.01     21.01    ]
 ...
 [52.       51.989998 51.98     ... 41.45     41.44     41.44    ]
 [52.01     52.       51.989998 ... 41.46     41.46     41.46    ]
 [52.02     52.02     52.01     ... 41.48     41.469997 41.469997]]
(3232, 3200)


In [ ]:
prediction_v = v03.createVariable('prediction', np.float64, ('longitude','latitude'))
data_arr = np.random.uniform(low=280,high=330,size=(3232,3200))
prediction_v[:,:] = data_arr # Appends data along unlimited dimension
print("-- Wrote data, prediction_v.shape is now ", prediction_v.shape)
print("-- Min/Max values:", prediction_v[:,:].min(), prediction_v[:,:].max())


ValueError: ignored

In [ ]:
v03_sza_t = np.reshape(v03_sza, (-1, 1))
print(v03_sza_t.shape)
v03_sza_o = np.reshape(v03_sza_t, (3232,-1))
print(v03_sza_o.shape)
print(v03_sza_o)

(10342400, 1)
(3232, 3200)
[[36.85     36.84     36.829998 ... 20.99     20.98     20.98    ]
 [36.86     36.85     36.829998 ... 21.       21.       21.      ]
 [36.87     36.86     36.84     ... 21.01     21.01     21.01    ]
 ...
 [52.       51.989998 51.98     ... 41.45     41.44     41.44    ]
 [52.01     52.       51.989998 ... 41.46     41.46     41.46    ]
 [52.02     52.02     52.01     ... 41.48     41.469997 41.469997]]


Step 4: Now load VIIRS Observation file (VNP02MOD)

In [ ]:
v02 = Dataset(v02_file,'r')
#v02 = h5py.File(v02_file,'r')
#suppose you are using all of the 16 bands
#create a placeholder for all of these datasets
#you can also use 16 different variables if you want
v02group = v02['/observation_data']
v02_datasets = ['M01','M02','M03','M04','M05','M06','M07','M08',
                'M09','M10','M11','M12','M13','M14','M15', 'M16']
sw_bands = ['M01','M02','M03','M04','M05','M06','M07','M08','M09','M10','M11']
ir_bands = ['M12','M13','M14','M15', 'M16']

n_channel = len(v02_datasets) #16 channels

v02_obs = np.full([v03_lon.shape[0],v03_lon.shape[1],n_channel],fill_value=np.nan)

for i, v02_dataset in zip(np.arange(n_channel),v02_datasets):
    #sometimes some channels are not included, 
    #for example some sw bands are not included during nighttime
    #so here we need to check
    if (v02_dataset in v02group.variables.keys()):
        #for shortwave first
        if (v02_dataset in sw_bands):
            v02_data = v02['/observation_data/'+v02_dataset][:]
            #note that netcdf automatically apply data scale and offset to original data
            #v02_data_scale = v02['/observation_data/'+v02_dataset].scale_factor
            #v02_data_offset = v02['/observation_data/'+v02_dataset].add_offset
            v02_obs[:,:,i] = v02_data
        #then for longwave bands
        #for ir we need to convert radiance to brightness temperature
        if (v02_dataset in ir_bands):
            v02_data = v02['/observation_data/'+v02_dataset][:]
            v02_data_scale = v02['/observation_data/'+v02_dataset].scale_factor
            v02_data_offset = v02['/observation_data/'+v02_dataset].add_offset
            v02_data_lut = v02['/observation_data/'+v02_dataset+'_brightness_temperature_lut'][:]
            #note that netcdf automatically apply data scale and offset to original data
            #to convert it back to brightness temperature, we need to do the following steps
            temp = (v02_data - v02_data_offset) / v02_data_scale
            v02_obs[:,:,i] = v02_data_lut[np.int32(np.round(temp))]        
v02.close()

#here is an example of the 500th row, 500th col pixel
print (v02_obs[499,499,:])
#here is another example of the 500th row, 500th col pixel
print (v02_obs[0,0,:])

#for shortwave bands, you need to remove 65533 values,
#for ir bands, you need to remove -999.9 values,

v02_obs[v02_obs==65533] = np.nan
v02_obs[v02_obs<0] = np.nan

#here is an example of the 500th row, 500th col pixel
print (v02_obs[499,499,:])
#here is another example of the 500th row, 500th col pixel
print (v02_obs[0,0,:])
print (v02_obs.shape)

[5.91716051e-01 5.91816008e-01 5.85658550e-01 5.58389783e-01
 6.01851881e-01 1.38183028e-01 6.45014048e-01 6.07989311e-01
 1.55935704e-03 5.28422117e-01 3.95916760e-01 3.14150543e+02
 2.86308167e+02 2.73129303e+02 2.76499084e+02 2.76012756e+02]
[65533.         65533.         65533.         65533.
 65533.         65533.         65533.         65533.
 65533.         65533.         65533.          -999.90002441
  -999.90002441  -999.90002441  -999.90002441  -999.90002441]
[5.91716051e-01 5.91816008e-01 5.85658550e-01 5.58389783e-01
 6.01851881e-01 1.38183028e-01 6.45014048e-01 6.07989311e-01
 1.55935704e-03 5.28422117e-01 3.95916760e-01 3.14150543e+02
 2.86308167e+02 2.73129303e+02 2.76499084e+02 2.76012756e+02]
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
(3232, 3200, 16)


In [ ]:
print(v02_obs)

[[[           nan            nan            nan ...            nan
              nan            nan]
  [           nan            nan            nan ...            nan
              nan            nan]
  [           nan            nan            nan ...            nan
              nan            nan]
  ...
  [           nan            nan            nan ...            nan
              nan            nan]
  [           nan            nan            nan ...            nan
              nan            nan]
  [           nan            nan            nan ...            nan
              nan            nan]]

 [[           nan            nan            nan ...            nan
              nan            nan]
  [           nan            nan            nan ...            nan
              nan            nan]
  [           nan            nan            nan ...            nan
              nan            nan]
  ...
  [           nan            nan            nan ...            nan
          

In [ ]:
print (v02_obs.shape)
print(v03_vaa.shape)
v02_obs = np.nan_to_num(v02_obs)
viirs_data = np.stack((v03_sza, v03_saa, v03_vza, v03_vaa, v02_obs[:, :, 0], v02_obs[:, :, 1], v02_obs[:, :, 2], v02_obs[:, :, 3], v02_obs[:, :, 4], v02_obs[:, :, 5], v02_obs[:, :, 6], v02_obs[:, :, 7], v02_obs[:, :, 8], v02_obs[:, :, 9], v02_obs[:, :, 10], v02_obs[:, :, 11], v02_obs[:, :, 12], v02_obs[:, :, 13], v02_obs[:, :, 14], v02_obs[:, :, 15],v03_lat, v03_lon), axis=-1)
print(viirs_data.shape)
print(viirs_data)

# print(viirs_data[0:10, 0:5])
# print(viirs_data[0:10, 13:15])
# print(viirs_data[0:10, 15:])

(3232, 3200, 16)
(3232, 3200)
(3232, 3200, 22)
[[[  3685.         -12352.           7013.         ...      0.
       48.95002747   -176.01417542]
  [  3684.         -12355.           7009.         ...      0.
       48.95079422   -176.03709412]
  [  3683.         -12358.           7006.         ...      0.
       48.95155334   -176.05996704]
  ...
  [  2099.          16321.           7000.         ...      0.
       42.99634933    144.72705078]
  [  2098.          16316.           7003.         ...      0.
       42.98989868    144.70953369]
  [  2098.          16311.           7006.         ...      0.
       42.98312759    144.69116211]]

 [[  3686.         -12354.           7013.         ...      0.
       48.96475983   -176.01304626]
  [  3685.         -12357.           7009.         ...      0.
       48.96551514   -176.03598022]
  [  3683.         -12360.           7006.         ...      0.
       48.96626663   -176.05886841]
  ...
  [  2100.          16319.           7000.      

In [ ]:
print(viirs_data[0,0, :])
print(viirs_data[499, 499, :])

print(viirs_data[3231, 3199, :])


[  3685.         -12352.           7013.          -8702.
      0.              0.              0.              0.
      0.              0.              0.              0.
      0.              0.              0.              0.
      0.              0.              0.              0.
     48.95002747   -176.01417542]
[ 3.52100000e+03 -1.37310000e+04  5.63000000e+03 -9.25200000e+03
  5.91716051e-01  5.91816008e-01  5.85658550e-01  5.58389783e-01
  6.01851881e-01  1.38183028e-01  6.45014048e-01  6.07989311e-01
  1.55935704e-03  5.28422117e-01  3.95916760e-01  3.14150543e+02
  2.86308167e+02  2.73129303e+02  2.76499084e+02  2.76012756e+02
  5.22094803e+01  1.76011398e+02]
[ 4147.         14166.          7012.          4323.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
    59.77523422   123.23497009]


In [ ]:
viirs_data = np.reshape(viirs_data, ( -1, 22))
print(viirs_data)

[[  3685.         -12352.           7013.         ...      0.
      48.95002747   -176.01417542]
 [  3684.         -12355.           7009.         ...      0.
      48.95079422   -176.03709412]
 [  3683.         -12358.           7006.         ...      0.
      48.95155334   -176.05996704]
 ...
 [  4148.          14173.           7006.         ...      0.
      59.79627228    123.27410889]
 [  4147.          14169.           7009.         ...      0.
      59.78581238    123.25465393]
 [  4147.          14166.           7012.         ...      0.
      59.77523422    123.23497009]]


In [ ]:
print(viirs_data[0, :])
print(viirs_data[-1, :])
print(viirs_data[249000, :])

[  3685.         -12352.           7013.          -8702.
      0.              0.              0.              0.
      0.              0.              0.              0.
      0.              0.              0.              0.
      0.              0.              0.              0.
     48.95002747   -176.01417542]
[ 4147.         14166.          7012.          4323.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
    59.77523422   123.23497009]
[ 2.30700000e+03 -1.79340000e+04  5.38200000e+03  6.87600000e+03
  5.29661655e-01  5.16846895e-01  5.01813114e-01  4.66667593e-01
  4.86659348e-01  1.45679936e-01  5.04651964e-01  4.44996536e-01
  4.11430374e-02  2.55774558e-01  2.41900265e-01  2.85285614e+02
  2.66763611e+02  2.57345825e+02  2.58551788e+02  2.56806458e+02
  4.57748718e+01  1.51418228e+02]


Now you are ready to go.